In [ ]:
import pendulum

import tasks_alpaca
# dag
path_to_json = Path().cwd() / "data" / "tickers.json"

tickers = tasks_alpaca.get_tickers(path_to_json)

login_minio, password_minio = get_minio_credentials()
host = "localhost"
client_minio = Minio(
    endpoint=f"{host}:9000",
    access_key=login,
    secret_key=password,
    secure=False
    )

## this is a dag within a dag
#get bars
bars, is_next_page = tasks_alpaca.get_bars(
    tickers_to_search=tickers[:10],
    time_frame="1Min",
    date_start=pendulum.today().date().isoformat(),
    date_end=pendulum.today().date().isoformat())
# save bars to bucket
upload_task = upload_to_bucket(source_file=bars, bucket_name="airflowlearning.testdag", client=client_minio, destination_file):

In [3]:
import pendulum
pendulum.today().date().isoformat()

'2025-12-15'